In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
import os
import tqdm
from Libraries import Crawl

In [ ]:
# --- CONFIGURATION ---
BASE_URL = "https://vnexpress.net"
DATA_DIR = "Database"
URLS_FILE = os.path.join(DATA_DIR, "vnexpress_articles.jsonl")
MIN_YEAR = 2020
MAX_WORDS = 1000
TARGET_ARTICLES_PER_SUBTYPE = 33
MAX_CONCURRENT_WORKERS = 10
REQUEST_TIMEOUT = 30

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

TYPE_DICT = {
    'thoi-su': ['chinh-tri', 'dan-sinh', 'giao-thong'],
    'the-gioi': ['tu-lieu', 'phan-tich', 'quan-su'],
    'kinh-doanh': ['quoc-te', 'doanh-nghiep', 'chung-khoan'],
    'khoa-hoc': ['tin-tuc', 'phat-minh', 'the-gioi-tu-nhien'],
    'giai-tri': ['gioi-sao', 'sach', 'phim', 'nhac'],
    'the-thao': ['bong-da', 'tennis', 'cac-mon-khac'],
    'giao-duc': ['tin-tuc', 'tuyen-sinh', 'du-hoc'],
    'suc-khoe': ['tin-tuc', 'cac-benh', 'song-khoe'],
}

VIETNAMESE_DAYS = {
    "Chủ nhật": "Sunday", "Thứ hai": "Monday", "Thứ ba": "Tuesday",
    "Thứ tư": "Wednesday", "Thứ năm": "Thursday", "Thứ sáu": "Friday",
    "Thứ bảy": "Saturday"
}

In [ ]:
"""Hàm chính điều phối việc crawl dữ liệu."""
os.makedirs(DATA_DIR, exist_ok=True)

with Crawl.create_session() as session:
    print("Bước 1: Thu thập tất cả URLs...")
    all_article_urls = []
    for category, sub_categories in tqdm.tqdm(TYPE_DICT.items(), desc="Categories"):
        for sub_cat in sub_categories:
            all_article_urls.extend(Crawl.get_article_urls(session, category, sub_cat))
    
    print(f"Tổng cộng thu được {len(all_article_urls)} URLs.")
    print("\nBước 2: Bắt đầu crawl chi tiết bài báo...")
    all_articles = []
    with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_WORKERS) as executor:
        future_to_url = {executor.submit(Crawl.scrape_article_details, session, url): url for url in all_article_urls}
        progress_bar = tqdm.tqdm(as_completed(future_to_url), total=len(all_article_urls), desc="Crawling Articles")
        for future in progress_bar:
            if result := future.result():
                all_articles.append(result)

print(f"\nBước 3: Lưu {len(all_articles)} bài báo vào file...")
with open(URLS_FILE, "w", encoding="utf-8") as f:
    for article in all_articles:
        f.write(json.dumps(article, ensure_ascii=False) + "\n")
        
print(f"Hoàn thành! Dữ liệu đã lưu tại {URLS_FILE}")